In [2]:
%run 10_kenlm_common.ipynb

In [5]:
import os
input_dataset = "dataset_phonemes.txt"
output_directory = "output_directory/"
output_model_base = "output_model.klm"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Ngram size options
order_trigram = 3
order_unigram = 1

# Train

In [6]:
!kenlm/build/bin/lmplz -o 3    --text dataset_phonemes.txt     --arpa output_directory/output_model.klm_trigram.arpa     --discount_fallback --skip_symbols|     kenlm/build/bin/build_binary     -T /dev/stdin output_directory/output_model.klm_trigram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_inference_on_cluster/dataset_phonemes.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 635344 types 42
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:504 2:37367230464 3:70063554560
Substituting fallback discounts for order 0: D1=0.5 D2=1 D3+=1.5
Statistics:
1 42 D1=0.5 D2=1 D3+=1.5
2 1280 D1=0.420074 D2=0.998284 D3+=1.12999
3 16435 D1=0.483725 D2=1.009 D3+=1.52672
Memory estimate for binary LM:
type     kB
probing 319 assuming -p 1.5
probing 327 assuming -r models -p 1.5
trie     88 without quantization
trie     37 assuming -q 8 -b 8 quantization 
trie     87 assuming -a 22 array pointer compression
trie     37 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sort

In [9]:
!kenlm/build/bin/build_binary\
    output_directory/output_model.klm_trigram.arpa \
    output_directory/output_model.klm_trigram.bin

kenlm/build/bin/build_binary: error while loading shared libraries: libboost_program_options-mt.so.1.53.0: cannot open shared object file: No such file or directory


# compress model file

In [8]:
!kenlm/build/bin/build_binary\
    output_directory/output_model.klm_trigram.arpa \
    output_directory/output_model.klm_trigram.klm

Reading output_directory/output_model.klm_trigram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [21]:
arpa_path = "output_directory/output_model.klm_trigram.arpa"
vocabulary = get_vocabulary_from_arpa(arpa_path)
# Print or use the vocabulary as needed
print("Vocabulary:", vocabulary)
# Tokens to add
new_tokens = ["<pad>", "<sil>", "<spn>"]
# Tokens to exclude
tokens_to_exclude = ["<s>", "</s>"]
# Add tokens to the ARPA file excluding the specified tokens
add_tokens_to_arpa(arpa_path, new_tokens, tokens_to_exclude)
vocabulary = get_vocabulary_from_arpa(arpa_path)

# Print or use the vocabulary as needed
print("Vocabulary:", vocabulary)

Vocabulary: {'2=1280', '3=16435', '1=42'}
Vocabulary: {'2=1280', '3=16435', '1=42'}
